In [6]:
import json

def clean_sleep_data(input_file_path, output_file_path):
    with open(input_file_path, 'r') as f:
        data = json.load(f)

    # keys to remove
    keys_to_remove = ["heart_rate", "awake_time", "bedtime_end_delta", "average_breath_variation",
                      "average_heart_rate", "average_hrv", "contributors", "efficiency",
                      "got_ups", "heart_rate", "hrv", "latency", "lowest_heart_rate",
                      "lowest_heart_rate_time_offset", "movement_30_sec", "period",
                      "readiness", "readiness_score_delta", "restless_periods"
                      "score", "segment_state", "sleep_algorithm_version",
                      "sleep_midpoint", "sleep_score_delta", "type", "wake_ups",
                      "sleep_phase_5_min", "timezone", "bedtime_start_delta",
                      "bedtime_end_delta", "midpoint_at_delta", "restless_periods",
                      "score", "time_in_bed", "total_sleep_duration", "restless"]

    # Iterate over each entry and remove irrelevant keys
    for entry in data['sleep']:
        for key in keys_to_remove:
            if key in entry:
                del entry[key]

    # Write cleaned data to a new JSON file
    with open(output_file_path, 'w') as f:
        json.dump(data, f, indent=2)

input_file_path = 'oura_sleepData.json'
output_file_path = 'cleaned_oura_SleepData.json'
clean_sleep_data(input_file_path, output_file_path)


In [7]:
import json

def clean_sleep_data_garmin(input_file_path, output_file_path):
    with open(input_file_path, 'r') as f:
        data = json.load(f)

    # keys to remove
    keys_to_remove = ["sleepWindowConfirmationType", "unmeasurableSeconds",
                      "lowestRespiration", "highestRespiration", "retro",
                      "awakeSleepSeconds"]

    # Iterate over each entry and remove unnecessary keys
    for entry in data:
        for key in keys_to_remove:
            if key in entry:
                del entry[key]

    # Write modified data to a new JSON file
    with open(output_file_path, 'w') as f:
        json.dump(data, f, indent=2)

input_file_path = 'garmin_sleepData.json'
output_file_path = 'cleaned_garmin_sleepData.json'
clean_sleep_data_garmin(input_file_path, output_file_path)
